In [4]:
!pip install sqlalchemy

# SQLalchemy

https://docs.sqlalchemy.org/en/20/orm/queryguide/

## ORM - Object Relational Mapping

## Mapping

In [58]:
from typing import List, Optional
from sqlalchemy import ForeignKey, String
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column, relationship


class Base(DeclarativeBase):
    pass


class Faculty(Base):
    __tablename__ = "faculty"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    num: Mapped[int]

    courses: Mapped[List["Course"]] = relationship(back_populates="faculty", cascade="all, delete-orphan")

    def __repr__(self) -> str:
        return f"Faculty(id={self.id!r}, name={self.name!r}, num={self.num!r})"


class Course(Base):
    __tablename__ = "course"
    id: Mapped[int] = mapped_column(primary_key=True)
    num: Mapped[int]

    faculty_id: Mapped[int] = mapped_column(ForeignKey("faculty.id"))
    faculty: Mapped["Faculty"] = relationship(back_populates="courses")

    def __repr__(self) -> str:
        return f"Course(id={self.id!r}, num={self.num!r}, faculty={self.faculty.name!r})"


## Engine

In [61]:
from sqlalchemy import create_engine

DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'students'
DB_USER = 'postgres'
DB_PASSWORD = 'root'

engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

## Direct SQL; Connection

In [64]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select * from faculty"))
    print(result.all())

[(1, 51, 'ПИНЖ'), (2, 52, 'ФИИТ'), (3, 53, 'МЕХТЕХ'), (5, 39, 'IT4'), (6, 905, 'Test5'), (7, 906, 'Test6'), (8, 901, 'Test1'), (9, 902, 'Test2'), (4, 38, 'IT31')]


## Direct SQL; Session

In [67]:
from sqlalchemy.orm import Session

stmt = text("SELECT id, name, num FROM faculty WHERE id > :var ORDER BY name")
with Session(engine) as session:
    result = session.execute(stmt, {"var": 2})
    for row in result:
        print(f"id: {row.id}  name: {row.name}  num: {row.num}")

id: 4  name: IT31  num: 38
id: 5  name: IT4  num: 39
id: 8  name: Test1  num: 901
id: 9  name: Test2  num: 902
id: 6  name: Test5  num: 905
id: 7  name: Test6  num: 906
id: 3  name: МЕХТЕХ  num: 53


# ORM

#### Logging

In [71]:
import logging
logging.basicConfig()

def log_on():
    logging.getLogger('sqlalchemy.engine').setLevel(logging.INFO)

def log_off():
    logging.getLogger('sqlalchemy.engine').setLevel(logging.ERROR)

log_on()

In [73]:
log_off()

## Select

In [76]:
from sqlalchemy import select

stmt = select(Faculty).where(Faculty.name.in_(["ФИИТ", "ПИНЖ"]))
print(stmt)

SELECT faculty.id, faculty.name, faculty.num 
FROM faculty 
WHERE faculty.name IN (__[POSTCOMPILE_name_1])


In [78]:
from sqlalchemy.orm import Session
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(Faculty).where(Faculty.name.in_(["ФИИТ", "ПИНЖ"]))
    for fac in session.scalars(stmt):
        print(fac)
        print(fac.courses)

Faculty(id=1, name='ПИНЖ', num=51)
[Course(id=1, num=11, faculty='ПИНЖ'), Course(id=2, num=12, faculty='ПИНЖ'), Course(id=3, num=13, faculty='ПИНЖ'), Course(id=4, num=14, faculty='ПИНЖ'), Course(id=5, num=15, faculty='ПИНЖ')]
Faculty(id=2, name='ФИИТ', num=52)
[Course(id=6, num=21, faculty='ФИИТ'), Course(id=7, num=22, faculty='ФИИТ'), Course(id=8, num=23, faculty='ФИИТ'), Course(id=9, num=24, faculty='ФИИТ'), Course(id=10, num=25, faculty='ФИИТ')]


In [80]:
fac.id

2

In [82]:
with Session(engine) as session:
    stmt = select(Course)
    for course in session.scalars(stmt):
        print(course, course.id, course.num, course.faculty.name)

Course(id=1, num=11, faculty='ПИНЖ') 1 11 ПИНЖ
Course(id=2, num=12, faculty='ПИНЖ') 2 12 ПИНЖ
Course(id=3, num=13, faculty='ПИНЖ') 3 13 ПИНЖ
Course(id=4, num=14, faculty='ПИНЖ') 4 14 ПИНЖ
Course(id=5, num=15, faculty='ПИНЖ') 5 15 ПИНЖ
Course(id=6, num=21, faculty='ФИИТ') 6 21 ФИИТ
Course(id=7, num=22, faculty='ФИИТ') 7 22 ФИИТ
Course(id=8, num=23, faculty='ФИИТ') 8 23 ФИИТ
Course(id=9, num=24, faculty='ФИИТ') 9 24 ФИИТ
Course(id=10, num=25, faculty='ФИИТ') 10 25 ФИИТ
Course(id=11, num=31, faculty='МЕХТЕХ') 11 31 МЕХТЕХ
Course(id=12, num=32, faculty='МЕХТЕХ') 12 32 МЕХТЕХ
Course(id=13, num=33, faculty='МЕХТЕХ') 13 33 МЕХТЕХ
Course(id=14, num=34, faculty='МЕХТЕХ') 14 34 МЕХТЕХ
Course(id=15, num=35, faculty='МЕХТЕХ') 15 35 МЕХТЕХ
Course(id=16, num=389, faculty='IT31') 16 389 IT31


In [84]:
list(Course.__table__.columns)[0].name

'id'

#### to dict

In [87]:
import pandas as pd

with Session(engine) as session:
    stmt = select(Course)
    for course in session.scalars(stmt).all():
        print(dict((col.name, getattr(course, col.name)) for col in Course.__table__.columns))

{'id': 1, 'num': 11, 'faculty_id': 1}
{'id': 2, 'num': 12, 'faculty_id': 1}
{'id': 3, 'num': 13, 'faculty_id': 1}
{'id': 4, 'num': 14, 'faculty_id': 1}
{'id': 5, 'num': 15, 'faculty_id': 1}
{'id': 6, 'num': 21, 'faculty_id': 2}
{'id': 7, 'num': 22, 'faculty_id': 2}
{'id': 8, 'num': 23, 'faculty_id': 2}
{'id': 9, 'num': 24, 'faculty_id': 2}
{'id': 10, 'num': 25, 'faculty_id': 2}
{'id': 11, 'num': 31, 'faculty_id': 3}
{'id': 12, 'num': 32, 'faculty_id': 3}
{'id': 13, 'num': 33, 'faculty_id': 3}
{'id': 14, 'num': 34, 'faculty_id': 3}
{'id': 15, 'num': 35, 'faculty_id': 3}
{'id': 16, 'num': 389, 'faculty_id': 4}


#### Join

In [90]:
with Session(engine) as session:
    stmt = (
        select(Course)
            .join(Course.faculty)
            .where(Course.num == 21)
            .where(Faculty.name == "ФИИТ")
        )
    result = session.scalars(stmt).one()
    print(result)

Course(id=6, num=21, faculty='ФИИТ')


#### Limit, first, order_by, one, one_or_none

In [93]:
with Session(engine) as session:
    result = session.scalars(select(Faculty).order_by(Faculty.name).limit(1)).one_or_none()
    print(result)

Faculty(id=4, name='IT31', num=38)


#### execute vs scalars

In [96]:
with Session(engine) as session:
    query = select(Faculty.name, Faculty.id).order_by(Faculty.name)
    result = session.execute(query).first()
    print(result)

('IT31', 4)


In [98]:
with Session(engine) as session:
    query = select(Faculty.name, Faculty.id).order_by(Faculty.name)
    result = session.scalars(query).all()
    print(result)

['IT31', 'IT4', 'Test1', 'Test2', 'Test5', 'Test6', 'МЕХТЕХ', 'ПИНЖ', 'ФИИТ']


### Columns

In [103]:
from sqlalchemy import select
from sqlalchemy.orm import load_only

with Session(engine) as session:
    stmt = select(Faculty).options(load_only(Faculty.name))
    res = session.scalars(stmt)#.all()
    for fac in res:
        print(fac.name, fac.id)

ПИНЖ 1
ФИИТ 2
МЕХТЕХ 3
IT4 5
Test5 6
Test6 7
Test1 8
Test2 9
IT31 4


In [105]:
### НО!!! запрос неперечисленной колонки

In [107]:
from sqlalchemy import select
from sqlalchemy.orm import load_only

with Session(engine) as session:
    stmt = select(Faculty).options(load_only(Faculty.name))
    res = session.scalars(stmt).all()
    for fac in res:
        print(fac)

Faculty(id=1, name='ПИНЖ', num=51)
Faculty(id=2, name='ФИИТ', num=52)
Faculty(id=3, name='МЕХТЕХ', num=53)
Faculty(id=5, name='IT4', num=39)
Faculty(id=6, name='Test5', num=905)
Faculty(id=7, name='Test6', num=906)
Faculty(id=8, name='Test1', num=901)
Faculty(id=9, name='Test2', num=902)
Faculty(id=4, name='IT31', num=38)


#### _and, _or

In [112]:
from sqlalchemy import and_, or_

session = Session(engine)

query = select(Course).where(
        and_(
            or_(Course.num == 12, Course.num == 11),
            Faculty.id == Course.faculty_id,
        )
    )
print(query)
print('-' * 50)

result = session.scalars(query).all()
print(result)
session.close()

SELECT course.id, course.num, course.faculty_id 
FROM course, faculty 
WHERE (course.num = :num_1 OR course.num = :num_2) AND faculty.id = course.faculty_id
--------------------------------------------------
[Course(id=2, num=12, faculty='ПИНЖ'), Course(id=1, num=11, faculty='ПИНЖ')]


#### scalars vs scalar

In [115]:
from sqlalchemy import and_, or_

session = Session(engine)

query = select(Course).where(
        and_(
            or_(Course.num == 12, Course.num == 11),
            Faculty.id == Course.faculty_id,
        )
    )
print(query)
print('-' * 50)

result = session.scalar(query)
print(result)
session.close()

SELECT course.id, course.num, course.faculty_id 
FROM course, faculty 
WHERE (course.num = :num_1 OR course.num = :num_2) AND faculty.id = course.faculty_id
--------------------------------------------------
Course(id=2, num=12, faculty='ПИНЖ')


### Group by

In [118]:
from sqlalchemy import func

session = Session(engine)

query = (
    select(Faculty.name, func.count(Course.id).label("courses"))
    .join(Course)
    .group_by(Faculty.name)
    .having(func.count(Course.id) > 1)
)

print(query)

result = session.execute(query).all()
print(result)

session.close()


SELECT faculty.name, count(course.id) AS courses 
FROM faculty JOIN course ON faculty.id = course.faculty_id GROUP BY faculty.name 
HAVING count(course.id) > :count_1
[('ФИИТ', 5), ('МЕХТЕХ', 5), ('ПИНЖ', 5)]


#### count

In [121]:
print(select(func.count()).select_from(Faculty))

SELECT count(*) AS count_1 
FROM faculty


### Другой вар запроса

In [124]:
faculty_table = Faculty.__table__

stmt = select(faculty_table.c.id, faculty_table.c.name).where(faculty_table.c.name == "ФИИТ")
print(stmt)

session = Session(engine)

result = session.execute(stmt)
print(result.all())


session.close()

SELECT faculty.id, faculty.name 
FROM faculty 
WHERE faculty.name = :name_1
[(2, 'ФИИТ')]


## Insert

In [127]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    f1 = Faculty(name="IT3", num=38)
    f2 = Faculty(name="IT4", num=39)
    session.add_all([f1, f2])
    session.commit()

IntegrityError: (psycopg2.errors.UniqueViolation) ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "faculty_num_uniq"
DETAIL:  Ключ "(num)=(38)" уже существует.

[SQL: INSERT INTO faculty (name, num) SELECT p0::VARCHAR, p1::INTEGER FROM (VALUES (%(name__0)s, %(num__0)s, 0), (%(name__1)s, %(num__1)s, 1)) AS imp_sen(p0, p1, sen_counter) ORDER BY sen_counter RETURNING faculty.id, faculty.id AS id__1]
[parameters: {'name__0': 'IT3', 'num__0': 38, 'name__1': 'IT4', 'num__1': 39}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
with Session(engine) as session:
    stmt = select(Faculty)
    for fac in session.scalars(stmt):
        print(fac)

### Bulk Insert

In [ ]:
from sqlalchemy import insert

with Session(engine) as session:
    result = session.execute(
        insert(Faculty),
        [
            {"name": "Test5", "num": 905},
            {"name": "Test6", "num": 906},
        ],
    )
    session.commit()
    # for i in result:
    #     print(i)

#### Returning

In [132]:
from sqlalchemy import insert

with Session(engine) as session:
    result = session.execute(
        insert(Faculty).returning(Faculty),
        [
            {"name": "Test1", "num": 901},
            {"name": "Test2", "num": 902},
        ],
    )
    session.commit()
    for i in result:
        print(i)

IntegrityError: (psycopg2.errors.UniqueViolation) ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "faculty_num_uniq"
DETAIL:  Ключ "(num)=(901)" уже существует.

[SQL: INSERT INTO faculty (name, num) VALUES (%(name__0)s, %(num__0)s), (%(name__1)s, %(num__1)s) RETURNING faculty.id, faculty.name, faculty.num]
[parameters: {'name__0': 'Test1', 'num__0': 901, 'name__1': 'Test2', 'num__1': 902}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

## Update

### update one

In [136]:
session = Session(engine)

query = select(Faculty).where(Faculty.num == 38)
fac = session.scalars(query).one()
print(fac)

fac.name = 'IT31'
session.commit()

fac = session.scalars(query).one()
print(fac)

session.close()

Faculty(id=4, name='IT31', num=38)
Faculty(id=4, name='IT31', num=38)


In [138]:
from sqlalchemy import update

stmt = (
    update(user_table)
    .where(user_table.c.name == "patrick")
    .values(fullname="Patrick the Star")
)
print(stmt)

NameError: name 'user_table' is not defined

In [140]:
Faculty.__table__

Table('faculty', MetaData(), Column('id', Integer(), table=<faculty>, primary_key=True, nullable=False), Column('name', String(length=30), table=<faculty>, nullable=False), Column('num', Integer(), table=<faculty>, nullable=False), schema=None)

### update related

In [143]:
with Session(engine) as session:
    query = select(Faculty).where(Faculty.num == 38)
    fac = session.scalars(query).one()
    fac.courses.append(Course(num=389))
    session.commit()

IntegrityError: (psycopg2.errors.UniqueViolation) ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "course_num_uniq"
DETAIL:  Ключ "(num)=(389)" уже существует.

[SQL: INSERT INTO course (num, faculty_id) VALUES (%(num)s, %(faculty_id)s) RETURNING course.id]
[parameters: {'num': 389, 'faculty_id': 4}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [145]:
with Session(engine) as session:
    stmt = (
        select(Course)
            .join(Course.faculty)
            .where(Faculty.num == 38)
        )
    result = session.scalars(stmt).all()
    print(result)

[Course(id=16, num=389, faculty='IT31')]


### Bulk Update

## Deletion

In [150]:
with Session(engine) as session:
    c1 = session.get(Course, 25)
    session.delete(c1)
    session.commit()

UnmappedInstanceError: Class 'builtins.NoneType' is not mapped

#### Bulk delete

In [153]:
with Session(engine) as session:
    stmt = Faculty.__table__.delete().where(Faculty.id > 10)
    session.execute(stmt)
    session.commit()

1. Сделать ORM таблички к остальным таблицам (Group, Student, Mark, Teacher, etc.)
2. Проверить правильность созданных классов с помощью ORM запросов select, используя
https://docs.sqlalchemy.org/en/20/orm/queryguide/select.html
    1) all(), one(), first(), one_or_none()
    2) where()
    3) order_by, desc
    4) limit, offset, count
    5) where, and_, or_
    6) count, distinct
    7) join, group_dy
    8) exists, см. https://docs.sqlalchemy.org/en/20/core/selectable.html#sqlalchemy.sql.expression.exists
    9) like, not like (подсмотреть в примере https://docs.sqlalchemy.org/en/20/tutorial/data_select.html#orm-entity-subqueries-ctes)
3. вывести 1-2 результатов таких запросов как датафрейм
4. Выбрать студентов одной группы, и тем, у кого нет оценок, добавить рандомные оценки от 2 до 5 по одному предмету за одну дату.
см. https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-queryguide-bulk-update
5. Выбрать оценки "2", добавленные в п.4, и исправить их на рандомно "3" или "4".
6. Удалить 2 рандомные оценки из п.4

In [1]:
from typing import List, Optional
from sqlalchemy import ForeignKey, String
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column, relationship


class Base(DeclarativeBase):
    pass


class Faculty(Base):
    __tablename__ = "faculty"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    num: Mapped[int]

    courses: Mapped[List["Course"]] = relationship(back_populates="faculty", cascade="all, delete-orphan")

    def __repr__(self) -> str:
        return f"Faculty(id={self.id!r}, name={self.name!r}, num={self.num!r})"


class Course(Base):
    __tablename__ = "course"
    id: Mapped[int] = mapped_column(primary_key=True)
    num: Mapped[int]

    faculty_id: Mapped[int] = mapped_column(ForeignKey("faculty.id"))
    faculty: Mapped["Faculty"] = relationship(back_populates="courses")

    groups: Mapped[List["Group"]] = relationship(back_populates="course", cascade="all, delete-orphan")
    def __repr__(self) -> str:
        return f"Course(id={self.id!r}, num={self.num!r}, faculty={self.faculty.name!r})"

class Group(Base):
    __tablename__ = "groups"
    id: Mapped[int] = mapped_column(primary_key=True)
    num: Mapped[int]
    commercial: Mapped[bool]
    course_id: Mapped[int] = mapped_column(ForeignKey("course.id"))
    course: Mapped["Course"] = relationship(back_populates="groups")

    student: Mapped[List["Student"]] = relationship(back_populates="groups", cascade="all, delete-orphan")
    def __repr__(self) -> str:
        return f"Group(id={self.id!r}, num={self.num!r}, course={self.course.num!r}, commercial={self.commercial!r})"

class Student(Base):
    __tablename__ = "student"
    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30))
    last_name: Mapped[str] = mapped_column(String(30))
    middle_name: Mapped[str] = mapped_column(String(30))
    date_of_birth: Mapped[str] = mapped_column(String(30))
    inn: Mapped[int]
    gender: Mapped[str] = mapped_column(String(30))
    
    groups_id: Mapped[int] = mapped_column(ForeignKey("groups.id"))
    groups: Mapped["Group"] = relationship(back_populates="student")

    marks: Mapped[List["Marks"]] = relationship(back_populates="student", cascade="all, delete-orphan")
    def __repr__(self) -> str:
        return f"Student(id={self.id!r}, name={self.first_name!r} {self.middle_name!r} {self.last_name!r}, group={self.groups.num!r}, date_of_birth={self.date_of_birth!r}, inn={self.inn!r}, gender={self.gender!r})"

class Subject(Base):
    __tablename__ = "subjects"
    id: Mapped[int] = mapped_column(primary_key=True)
    subject: Mapped[str] = mapped_column(String(30))
    
    marks: Mapped[List["Marks"]] = relationship(back_populates="subjects", cascade="all, delete-orphan")
    def __repr__(self) -> str:
        return f"Subject(id={self.id!r}, name={self.subject!r}"

class Marks(Base):
    __tablename__ = "marks"
    id: Mapped[int] = mapped_column(primary_key=True)
    mark: Mapped[int]
    subject_id: Mapped[int] = mapped_column(ForeignKey("subjects.id"))
    subjects: Mapped["Subject"] = relationship(back_populates="marks")
    student_id: Mapped[int] = mapped_column(ForeignKey("student.id"))
    student: Mapped["Student"] = relationship(back_populates="marks")
    def __repr__(self) -> str:
        return f"Mark(id={self.id!r}, mark={self.mark!r}, subject={self.subjects.subject}, student={self.student.first_name!r} {self.student.middle_name!r} {self.student.last_name!r}"

In [3]:
from sqlalchemy import create_engine

DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'students'
DB_USER = 'postgres'
DB_PASSWORD = 'root'

engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [5]:
from sqlalchemy.orm import Session
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(Faculty)
    for fac in session.scalars(stmt):
        print(fac)

Faculty(id=1, name='ПИНЖ', num=51)
Faculty(id=2, name='ФИИТ', num=52)
Faculty(id=3, name='МЕХТЕХ', num=53)
Faculty(id=5, name='IT4', num=39)
Faculty(id=6, name='Test5', num=905)
Faculty(id=7, name='Test6', num=906)
Faculty(id=8, name='Test1', num=901)
Faculty(id=9, name='Test2', num=902)
Faculty(id=4, name='IT31', num=38)


In [7]:
from sqlalchemy import select

stmt = select(Faculty)
session.execute(stmt).all()

[(Faculty(id=1, name='ПИНЖ', num=51),),
 (Faculty(id=2, name='ФИИТ', num=52),),
 (Faculty(id=3, name='МЕХТЕХ', num=53),),
 (Faculty(id=5, name='IT4', num=39),),
 (Faculty(id=6, name='Test5', num=905),),
 (Faculty(id=7, name='Test6', num=906),),
 (Faculty(id=8, name='Test1', num=901),),
 (Faculty(id=9, name='Test2', num=902),),
 (Faculty(id=4, name='IT31', num=38),)]

In [9]:
stmt = select(Student)
session.execute(stmt).all()

[(Student(id=4, name='Иван' 'Иванович' 'Иванов', group=111, date_of_birth=datetime.date(2006, 2, 2), inn=525677889912, gender='М'),),
 (Student(id=5, name='Никита' 'Олегович' 'Никитин', group=111, date_of_birth=datetime.date(2006, 3, 3), inn=525677889913, gender='М'),),
 (Student(id=6, name='Марина' 'Ивановна' 'Маринина', group=111, date_of_birth=datetime.date(2006, 3, 4), inn=525677889914, gender='Ж'),),
 (Student(id=7, name='Жанна' 'Алексеевна' 'Сергеева', group=111, date_of_birth=datetime.date(2006, 1, 1), inn=525677889915, gender='Ж'),),
 (Student(id=8, name='Кирилл' 'Кириллович' 'Сергеев', group=112, date_of_birth=datetime.date(2006, 11, 5), inn=525677889916, gender='М'),),
 (Student(id=9, name='Федя' 'Потапович' 'Колбаскин', group=112, date_of_birth=datetime.date(2006, 10, 10), inn=525677889917, gender='М'),),
 (Student(id=10, name='Ольга' 'Петровна' 'Петрова', group=112, date_of_birth=datetime.date(2006, 5, 6), inn=525677889918, gender='Ж'),),
 (Student(id=11, name='Валя' 'Патри

In [11]:
session.execute(stmt).first()

(Student(id=4, name='Иван' 'Иванович' 'Иванов', group=111, date_of_birth=datetime.date(2006, 2, 2), inn=525677889912, gender='М'),)

In [13]:
session = Session(engine)

stmt = select(Student).where(Student.id == 32)
f = session.scalars(stmt).one()
print(f)

Student(id=32, name='Анна' 'Алексеевна' 'Одинцова', group=123, date_of_birth=datetime.date(2005, 10, 21), inn=525677889941, gender='Ж')


In [15]:
session = Session(engine)

stmt = select(Student).where(Student.id == 37)
fac = session.scalars(stmt).one_or_none()
print(fac)

None


In [17]:
stmt = select(Student).where(Student.id == 11)
session.execute(stmt).all()

[(Student(id=11, name='Валя' 'Патрикеевна' 'Изенштейн', group=112, date_of_birth=datetime.date(2006, 12, 3), inn=525677889919, gender='Ж'),)]

In [19]:
stmt = select(Student).where(Student.groups_id == 1).order_by(Student.first_name)
session.execute(stmt).all()

[(Student(id=33, name='Алина' 'Александровна' 'Редькина', group=111, date_of_birth=datetime.date(2005, 5, 14), inn=52544444444, gender='Ж'),),
 (Student(id=38, name='Анна' 'Дмитриевна' 'Алексеева', group=111, date_of_birth=datetime.date(2006, 5, 1), inn=525677889999, gender=None),),
 (Student(id=41, name='Дмитрий' 'Алексеевич' 'Дмитриев', group=111, date_of_birth=datetime.date(2006, 8, 4), inn=525677889993, gender=None),),
 (Student(id=42, name='Екатерина' 'Сергеевна' 'Петрова', group=111, date_of_birth=datetime.date(2006, 9, 1), inn=525677889988, gender=None),),
 (Student(id=7, name='Жанна' 'Алексеевна' 'Сергеева', group=111, date_of_birth=datetime.date(2006, 1, 1), inn=525677889915, gender='Ж'),),
 (Student(id=4, name='Иван' 'Иванович' 'Иванов', group=111, date_of_birth=datetime.date(2006, 2, 2), inn=525677889912, gender='М'),),
 (Student(id=6, name='Марина' 'Ивановна' 'Маринина', group=111, date_of_birth=datetime.date(2006, 3, 4), inn=525677889914, gender='Ж'),),
 (Student(id=5, nam

In [21]:
from sqlalchemy import desc

stmt = select(Student).where(Student.groups_id == 2).order_by(desc(Student.first_name))
session.execute(stmt).all()

[(Student(id=9, name='Федя' 'Потапович' 'Колбаскин', group=112, date_of_birth=datetime.date(2006, 10, 10), inn=525677889917, gender='М'),),
 (Student(id=10, name='Ольга' 'Петровна' 'Петрова', group=112, date_of_birth=datetime.date(2006, 5, 6), inn=525677889918, gender='Ж'),),
 (Student(id=8, name='Кирилл' 'Кириллович' 'Сергеев', group=112, date_of_birth=datetime.date(2006, 11, 5), inn=525677889916, gender='М'),),
 (Student(id=12, name='Дмитрий' 'Осипович' 'Оз', group=112, date_of_birth=datetime.date(2006, 6, 7), inn=525677889920, gender='М'),),
 (Student(id=11, name='Валя' 'Патрикеевна' 'Изенштейн', group=112, date_of_birth=datetime.date(2006, 12, 3), inn=525677889919, gender='Ж'),)]

In [23]:
stmt = select(Student).where(Student.groups_id == 2).order_by(Student.first_name).limit(2)
session.execute(stmt).all()

[(Student(id=11, name='Валя' 'Патрикеевна' 'Изенштейн', group=112, date_of_birth=datetime.date(2006, 12, 3), inn=525677889919, gender='Ж'),),
 (Student(id=12, name='Дмитрий' 'Осипович' 'Оз', group=112, date_of_birth=datetime.date(2006, 6, 7), inn=525677889920, gender='М'),)]

In [25]:
stmt = select(Student).where(Student.groups_id == 2).order_by(Student.first_name).offset(3)
session.execute(stmt).all()

[(Student(id=10, name='Ольга' 'Петровна' 'Петрова', group=112, date_of_birth=datetime.date(2006, 5, 6), inn=525677889918, gender='Ж'),),
 (Student(id=9, name='Федя' 'Потапович' 'Колбаскин', group=112, date_of_birth=datetime.date(2006, 10, 10), inn=525677889917, gender='М'),)]

In [27]:
from sqlalchemy import func

stmt = select(func.count()).where(Student.groups_id == 2)
session.execute(stmt).all()

[(5,)]

In [29]:
from sqlalchemy import and_, or_

session = Session(engine)

stmt = select(Student).where(
        and_(
            or_(Student.groups_id == 1, Student.groups_id == 2),
            Student.gender == 'М',
        )
    )
session.execute(stmt).all()

[(Student(id=4, name='Иван' 'Иванович' 'Иванов', group=111, date_of_birth=datetime.date(2006, 2, 2), inn=525677889912, gender='М'),),
 (Student(id=5, name='Никита' 'Олегович' 'Никитин', group=111, date_of_birth=datetime.date(2006, 3, 3), inn=525677889913, gender='М'),),
 (Student(id=8, name='Кирилл' 'Кириллович' 'Сергеев', group=112, date_of_birth=datetime.date(2006, 11, 5), inn=525677889916, gender='М'),),
 (Student(id=9, name='Федя' 'Потапович' 'Колбаскин', group=112, date_of_birth=datetime.date(2006, 10, 10), inn=525677889917, gender='М'),),
 (Student(id=12, name='Дмитрий' 'Осипович' 'Оз', group=112, date_of_birth=datetime.date(2006, 6, 7), inn=525677889920, gender='М'),),
 (Student(id=3, name='Петр' 'Петрович' 'Петров', group=111, date_of_birth=datetime.date(2005, 1, 3), inn=525677889911, gender='М'),)]

In [31]:
from sqlalchemy import select, func
session = Session(engine)
stmt = select(func.distinct(Student.first_name)).where(Student.groups_id == 2)
session.execute(stmt).all()

[('Федя',), ('Кирилл',), ('Дмитрий',), ('Валя',), ('Ольга',)]

In [33]:
from sqlalchemy import select, func
session = Session(engine)

stmt = (
        select(Student)
            .join(Student.groups)
            .where(Student.id <= 20)
            .group_by(Student.id)
        )
session.execute(stmt).all()

[(Student(id=11, name='Валя' 'Патрикеевна' 'Изенштейн', group=112, date_of_birth=datetime.date(2006, 12, 3), inn=525677889919, gender='Ж'),),
 (Student(id=8, name='Кирилл' 'Кириллович' 'Сергеев', group=112, date_of_birth=datetime.date(2006, 11, 5), inn=525677889916, gender='М'),),
 (Student(id=19, name='Петр' 'Алексеевич' 'Первый', group=121, date_of_birth=datetime.date(2005, 5, 5), inn=525677889928, gender='М'),),
 (Student(id=4, name='Иван' 'Иванович' 'Иванов', group=111, date_of_birth=datetime.date(2006, 2, 2), inn=525677889912, gender='М'),),
 (Student(id=14, name='Гена' 'Герасимович' 'Офиногенов', group=113, date_of_birth=datetime.date(2006, 7, 30), inn=525677889923, gender='М'),),
 (Student(id=3, name='Петр' 'Петрович' 'Петров', group=111, date_of_birth=datetime.date(2005, 1, 3), inn=525677889911, gender='М'),),
 (Student(id=17, name='Зина' 'Леопольдовна' 'Турбина', group=113, date_of_birth=datetime.date(2006, 9, 1), inn=525677889926, gender='Ж'),),
 (Student(id=20, name='Данила'

In [35]:
from sqlalchemy import exists
session = Session(engine)

students_with_groups = session.query(Student).filter(
    exists().where(Student.groups_id == Group.id)
).all()

students_with_groups

[Student(id=4, name='Иван' 'Иванович' 'Иванов', group=111, date_of_birth=datetime.date(2006, 2, 2), inn=525677889912, gender='М'),
 Student(id=5, name='Никита' 'Олегович' 'Никитин', group=111, date_of_birth=datetime.date(2006, 3, 3), inn=525677889913, gender='М'),
 Student(id=6, name='Марина' 'Ивановна' 'Маринина', group=111, date_of_birth=datetime.date(2006, 3, 4), inn=525677889914, gender='Ж'),
 Student(id=7, name='Жанна' 'Алексеевна' 'Сергеева', group=111, date_of_birth=datetime.date(2006, 1, 1), inn=525677889915, gender='Ж'),
 Student(id=8, name='Кирилл' 'Кириллович' 'Сергеев', group=112, date_of_birth=datetime.date(2006, 11, 5), inn=525677889916, gender='М'),
 Student(id=9, name='Федя' 'Потапович' 'Колбаскин', group=112, date_of_birth=datetime.date(2006, 10, 10), inn=525677889917, gender='М'),
 Student(id=10, name='Ольга' 'Петровна' 'Петрова', group=112, date_of_birth=datetime.date(2006, 5, 6), inn=525677889918, gender='Ж'),
 Student(id=11, name='Валя' 'Патрикеевна' 'Изенштейн', g

In [37]:
subq = select(Subject).where(Subject.subject.like("%тика")).subquery()
query = select(subq)
session.execute(query).all()

[(1, 'Математика'),
 (2, 'Информатика'),
 (3, 'Дискретная математика'),
 (6, 'Высшая математика')]

In [39]:
subq = select(Subject).where(Subject.subject.notlike("%тика")).cte()
query = select(subq)
session.execute(query).all()

[(4, 'С++'), (5, 'Python')]

In [41]:
subq = select(Subject).where(~Subject.subject.notlike("%тика")).cte()
query = select(subq)
res = session.execute(query).all()
res

[(1, 'Математика'),
 (2, 'Информатика'),
 (3, 'Дискретная математика'),
 (6, 'Высшая математика')]

In [43]:
import pandas as pd
r = pd.DataFrame(res)
r

,id,subject
0,1,Математика
1,2,Информатика
2,3,Дискретная математика
3,6,Высшая математика


In [45]:
stmt = select(Student.id).where(Student.groups_id == 1).join(Student.marks, isouter=True).where(Marks.mark == None)
res = session.execute(stmt).all()

In [105]:
import random
from sqlalchemy import insert
session = Session(engine)
for r in res:
    k = r[0]
    i = random.randrange(2,6)
    j = random.randrange(1,7)
    session.execute(
        insert(Marks),
        [
            {"student_id": k, "subject_id": j, "mark": i},
        ],
    )

session.commit()

IntegrityError: (psycopg2.errors.UniqueViolation) ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "marks_student_id_uniq"
DETAIL:  Ключ "(student_id)=(2)" уже существует.

[SQL: INSERT INTO marks (mark, subject_id, student_id) VALUES (%(mark)s, %(subject_id)s, %(student_id)s) RETURNING marks.id]
[parameters: {'mark': 2, 'subject_id': 1, 'student_id': Decimal('2')}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [51]:
stmt = select(Marks)
session.execute(stmt).all()

[(Mark(id=1, mark=Decimal('3'), subject=Python, student='Петр' 'Петрович' 'Петров',),
 (Mark(id=2, mark=Decimal('5'), subject=Python, student='Иван' 'Иванович' 'Иванов',),
 (Mark(id=3, mark=Decimal('4'), subject=Python, student='Никита' 'Олегович' 'Никитин',),
 (Mark(id=4, mark=Decimal('4'), subject=Python, student='Марина' 'Ивановна' 'Маринина',),
 (Mark(id=5, mark=Decimal('3'), subject=Python, student='Жанна' 'Алексеевна' 'Сергеева',),
 (Mark(id=6, mark=Decimal('2'), subject=Математика, student='Кирилл' 'Кириллович' 'Сергеев',),
 (Mark(id=7, mark=Decimal('5'), subject=Информатика, student='Федя' 'Потапович' 'Колбаскин',),
 (Mark(id=8, mark=Decimal('5'), subject=Математика, student='Ольга' 'Петровна' 'Петрова',),
 (Mark(id=9, mark=Decimal('4'), subject=Математика, student='Валя' 'Патрикеевна' 'Изенштейн',),
 (Mark(id=10, mark=Decimal('3'), subject=Математика, student='Дмитрий' 'Осипович' 'Оз',),
 (Mark(id=11, mark=Decimal('3'), subject=Информатика, student='Остап' 'Изяславович' 'Остап

In [101]:
from sqlalchemy import select, update
import random

session = Session(engine)

for r in res:
    k = r[0]  
    print(k)
    stmt = select(Marks.mark).where(Marks.student_id == k)
    result = session.execute(stmt).all()
    if result:
        current_mark = result[0]
        print(current_mark)
        if current_mark == 2:
            new_mark = random.randrange(3, 5) 
            session.execute(
                update(Marks)
                .where(Marks.id == k)
                .values(mark=new_mark)
            )
            print(f"Обновлена оценка студента {k} с 2 на {new_mark}")
    
session.commit()

2


In [91]:
stmt = select(Marks)
session.execute(stmt).all()

[(Mark(id=1, mark=Decimal('3'), subject=Python, student='Петр' 'Петрович' 'Петров',),
 (Mark(id=2, mark=Decimal('5'), subject=Python, student='Иван' 'Иванович' 'Иванов',),
 (Mark(id=3, mark=Decimal('4'), subject=Python, student='Никита' 'Олегович' 'Никитин',),
 (Mark(id=4, mark=Decimal('4'), subject=Python, student='Марина' 'Ивановна' 'Маринина',),
 (Mark(id=5, mark=Decimal('3'), subject=Python, student='Жанна' 'Алексеевна' 'Сергеева',),
 (Mark(id=6, mark=Decimal('2'), subject=Математика, student='Кирилл' 'Кириллович' 'Сергеев',),
 (Mark(id=7, mark=Decimal('5'), subject=Информатика, student='Федя' 'Потапович' 'Колбаскин',),
 (Mark(id=8, mark=Decimal('5'), subject=Математика, student='Ольга' 'Петровна' 'Петрова',),
 (Mark(id=9, mark=Decimal('4'), subject=Математика, student='Валя' 'Патрикеевна' 'Изенштейн',),
 (Mark(id=10, mark=Decimal('3'), subject=Математика, student='Дмитрий' 'Осипович' 'Оз',),
 (Mark(id=11, mark=Decimal('3'), subject=Информатика, student='Остап' 'Изяславович' 'Остап

In [109]:
from sqlalchemy import delete
import random

session = Session(engine)

random_ids = random.sample(range(1, 31), 2)

for mark_id in random_ids:
    session.execute(
        delete(Marks)
        .where(Marks.id == mark_id)
    )
    print(f"Удаление оценки с ID {mark_id}")

session.commit()

Удаление оценки с ID 13
Удаление оценки с ID 7
